<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/google_markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Методика генерации меток тем с помощью ИПС


This script generates tags for a topic represented as a set of words using the Google search engine.

Этот скрипт генерирует метки для темы, представленной  в виде набора слов, используя поисковую систему Google.

In [ ]:
!pip install -q pymorphy3
!pip install - q gensim
import pymorphy3
from gensim import  models
import requests
from collections import Counter
import re
from string import punctuation

morph = pymorphy3.MorphAnalyzer(lang='ru')

In [ ]:
API_KEY = "ваш API KEY"
SEARCH_ENGINE_ID = "ваш SEARCH_ENGINE_ID"
GOOGLE_API_URL = "ваш GOOGLE_API_URL"

# Подготовка текста запроса

In [ ]:
lda_model = models.ldamodel.LdaModel.load('res.model')
queries = []
for n in range(0, lda_model.num_topics):
    queries.append(' '.join([term for term, wt in lda_model.show_topic(n)]))
query = queries[4]
query

'район посёлок коронавирус пневмония пациент госпитализировать исход подтвердить число инфекция'

# Получение меток

In [31]:
import requests
from collections import Counter
import re
from string import punctuation

query = "район коронавирус пневмония пациент госпитализировать исход подтвердить число инфекция"
query_words = {word.strip().lower() for word in query.split(" ")}

def preprocess_text(text):
    text = re.sub(r'\d+', '', text.lower())
    words = re.findall(r'[а-яё]+', text)
    return [
        morph.parse(w)[0].normal_form
        for w in words
        if len(w) > 2
    ]

def extract_ngrams(documents, n_range=(2, 3), top_n=5):
    ngrams_counter = Counter()

    for doc in documents:
        words = preprocess_text(doc)
        for n in range(n_range[0], n_range[1] + 1):
            ngrams_counter.update(
                [' '.join(words[i:i+n])
                 for i in range(len(words)-n+1)]
            )
    return sorted(
        ngrams_counter.items(),
        key=lambda x: (
            -x[1],  # Частота
            sum(word in x[0] for word in query_words),  # Совпадения с запросом
            -len(x[0].split())  # Предпочтение более длинным N-граммам
        )
    )[:top_n]

def get_search_results(query):
    url =  GOOGLE_API_URL
    params = {
        "key": API_KEY,
        "cx": SEARCH_ENGINE_ID,
        "q": query,
        "num": 10,
        "hl": "ru"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

try:
    response = get_search_results(query)
    documents = [
        f"{item['title']} {item.get('snippet', '')}"
        for item in response.get('items', [])
    ]

    top_phrases = extract_ngrams(documents, n_range=(2, 3))

    print("Топ релевантных русских словосочетаний:")
    for i, (phrase, count) in enumerate(top_phrases, 1):
        print(f"{i}. {phrase}")

except requests.exceptions.RequestException as e:
    print(f"Ошибка при выполнении запроса: {e}")

Топ релевантных русских словосочетаний:
1. госпитализировать пациент
2. случай заболевание
3. подтвердить заражение
4. двухсторонний пневмония
5. число случай
